In [1]:
import pandas as pd
import numpy as np
from HFC32_hmimCl_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.3
    init_kappa_1_2A = 0.02
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['hmimCl'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['hmimCl','hmimCl'].fix(0)
    m.fs.properties.PR_kappa_A['hmimCl', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'hmimCl'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['hmimCl', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_B['hmimCl', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_C['hmimCl', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_C['hmimCl', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'hmimCl'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['hmimCl'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'hmimCl'].fix(float(data['x_hmimCl']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['hmimCl'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['hmimCl', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['hmimCl', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'hmimCl'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'hmimCl'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['hmimCl', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'hmimCl']"]


In [3]:
data = pd.read_csv('R32_hmimCl.csv')

print(data)

   temperature  pressure  x_R32  x_hmimCl
0       298.15     50000  0.021     0.979
1       298.15    100000  0.044     0.956
2       298.15    200000  0.092     0.908
3       298.15    400000  0.186     0.814
4       298.15    600000  0.272     0.728
5       298.15    800000  0.345     0.655
6       298.15   1000000  0.410     0.590


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_hmimCl"] += data_["x_hmimCl"] * noise_x
    data_.loc[data_['x_hmimCl'] <= 0.0001, 'x_hmimCl'] = 0.0001
    data_.loc[data_['x_hmimCl'] >= 0.9999 , 'x_hmimCl'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.07684615320811901
2022-03-06 15:29:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:51 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a loc

2022-03-06 15:31:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.19132891182734565
fs.properties.PR_kappa_A[

2022-03-06 15:32:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.33312034134963564
fs.properties.PR_kappa_A[R32,hmimCl] = 0.016557557379662385
16
-0.026319265080529147
2022-03-06 15:32:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-

2022-03-06 15:33:24 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.3080142568580822
fs.properties.PR_kappa_A[R32,hmimCl] = 0.024147239651038203
23
0.09574625355114906
2022-03-06 15:33:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 

2022-03-06 15:34:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.3665444605099919
fs.properties.PR_kappa_A[R32,hmimCl] = 0.00499253315572849
30
-0.14675818760091452
2022-03-06 15:34:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 

...

2022-03-06 15:40:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.1889301621613508
fs.properties.PR_kappa_A[R32,hmimCl] = 0.031561319796493945
66
0.14600370340998195
2022-03-06 15:40:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:29 [INFO] idaes.init.fs.state_block: Property p

2022-03-06 15:41:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.1918069382340148
fs.properties.PR_kappa_A[R32,hmimCl] = 0.032095522463706685
73
-0.14124621065195136
2022-03-06 15:41:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:37 [INFO] idaes.init.fs.state_block: Property 

    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.2020117726990059
fs.properties.PR_kappa_A[R32,hmimCl] = 0.032028121335735224
80
0.06993134801086057
2022-03-06 15:42:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:58 [INFO] idaes.init.fs.state_block: Property p

2022-03-06 15:44:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:44:05 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:44:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:44:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.3491912965295853
fs.properties.PR_kappa_A[R32,hmimCl] = 0.011247685681372882
88
0.2656432324327879
2022-03-06 15:44:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:44:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:44:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 1

2022-03-06 15:45:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R32] = -0.19181081518108922
fs.properties.PR_kappa_A[R32,hmimCl] = 0.032168115846432026
95
-0.018379622195089592
2022-03-06 15:45:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:24 [INFO] idaes.init.fs.state_block: Propert

In [5]:
print(results_params)

[[-0.19149946  0.03190779]
 [-0.299868    0.02646202]
 [-0.29831508  0.02689626]
 [-0.32059068  0.02043937]
 [-0.30910567  0.02383218]
 [-0.36216235  0.00663112]
 [-0.32269532  0.01980129]
 [-0.30276962  0.02564476]
 [-0.19132891  0.03193546]
 [-0.19430432  0.03233803]
 [-0.18945968  0.03160369]
 [-0.188208    0.03150236]
 [-0.31454038  0.02224483]
 [-0.18828845  0.03150899]
 [-0.37029412  0.00355506]
 [-0.33312034  0.01655756]
 [-0.30709379  0.02441202]
 [-0.19180166  0.03226125]
 [-0.19073645  0.03169481]
 [-0.31755339  0.02135096]
 [-0.19002058  0.03164779]
 [-0.29844154  0.02686098]
 [-0.30801426  0.02414724]
 [-0.19133235  0.03179179]
 [-0.19130981  0.03195715]
 [-0.1913716   0.03181899]
 [-0.31341537  0.02257601]
 [-0.19174007  0.03206763]
 [-0.18677507  0.03136064]
 [-0.36654446  0.00499253]
 [-0.34990075  0.01100346]
 [-0.33129392  0.01713632]
 [-0.1939703   0.03234259]
 [-0.19139041  0.03183194]
 [-0.30279122  0.02563865]
 [-0.19161464  0.03203704]
 [-0.31056827  0.02340805]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_hmimCL_10_2.csv')